In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_score, f1_score, precision_score, recall_score
from skimage.metrics import structural_similarity as ssim
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import regularizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Lambda, GlobalAveragePooling2D, Conv2D
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

import random
import re
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import gc

2025-05-18 14:30:50.703332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-18 14:30:50.778535: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-18 14:30:50.810890: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 14:30:51.521536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU encontrada.")

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1747589473.927833    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747589478.680230    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747589478.680373    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# nome = 'PACS'
nome = 'Yildirim'
#nome = 'Todos'

imgs_path = 'Bases/Dataset' + nome

output_dir = "Results_Streamlit/CNN-MobileNet (5%)/"
os.makedirs(output_dir, exist_ok=True)

TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1

EPOCHS = 75
BATCH_SIZE = 32
SIGMA_BLUR = 0.8

HIGH_FREQ_THRESHOLD = 5
NUM_FOLD = 2

In [4]:
def get_next_filename(output_folder, base_name, type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    image_index = 0
    
    while True:
        output_filename = f"{base_name}_{image_index}.{type}"
        output_path = os.path.join(output_folder, output_filename)
        
        if not os.path.exists(output_path):
            return output_filename
            
        image_index += 1

In [5]:
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(0)) if match else 0

In [6]:
def sort_files_numerically(file_paths):
    return sorted(file_paths, key=lambda x: extract_number(x))

In [7]:
def read_dataset(path, jpg, png):
    print(f'Reading dataset...\n')
    
    img_type = []
    images = []
    image_paths = []

    if jpg:
        img_type.append('*.jpg')
    if png:
        img_type.append('*.png')

# Normal

    normal_path = os.path.join(path, 'Train', 'Normal')
    print(f'Reading Normal images from: {normal_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(normal_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# Kidney

    kidney_stone_path = os.path.join(path, 'Train', 'Kidney_stone')
    print(f'Reading Kidney_stone images from: {kidney_stone_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(kidney_stone_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# ...
    
    images = np.array(images)
    image_paths = np.array(image_paths)

    return images, image_paths

In [8]:
all_X, all_image_paths = read_dataset(path=imgs_path, jpg=True, png=True)

Reading dataset...

Reading Normal images from: Bases/DatasetYildirim/Train/Normal
Reading Kidney_stone images from: Bases/DatasetYildirim/Train/Kidney_stone


In [9]:
output_dir_shift = "All_fshift"
os.makedirs(output_dir_shift, exist_ok=True)

output_dir_spectrum = "All_freqs"
os.makedirs(output_dir_spectrum, exist_ok=True)

def freq_spec(image, threshold, add_noise):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)

    if add_noise:
        rows, cols = image.shape
        noise_size = int(np.sqrt(threshold * rows * cols))
        sigma = SIGMA_BLUR

        def gaussian_blur(size, sigma):
            ax = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
            xx, yy = np.meshgrid(ax, ax)
            kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    
            return kernel / np.sum(kernel)

        blur_kernel = gaussian_blur(noise_size, sigma)

        def blur_patch(spectrum, r_start, c_start):
            patch = spectrum[r_start:r_start+noise_size, c_start:c_start+noise_size]

            if patch.shape != blur_kernel.shape:
                return
            real_blurred = cv2.filter2D(np.real(patch), -1, blur_kernel)
            imag_blurred = cv2.filter2D(np.imag(patch), -1, blur_kernel)
            spectrum[r_start:r_start+noise_size, c_start:c_start+noise_size] = real_blurred + 1j * imag_blurred

        corner = np.random.randint(0, 4)
        if corner == 0:
            blur_patch(fshift, 0, 0)
        elif corner == 1:
            blur_patch(fshift, 0, cols - noise_size)
        elif corner == 2:
            blur_patch(fshift, rows - noise_size, 0)
        else:
            blur_patch(fshift, rows - noise_size, cols - noise_size)

    magnitude_spectrum_high = 20 * np.log(np.abs(fshift) + 1)

    return magnitude_spectrum_high

In [10]:
all_specs = []
all_labels = []

for i, img in enumerate(all_X):
    all_specs.append(freq_spec(img, HIGH_FREQ_THRESHOLD/100.0, add_noise=False))
    all_labels.append(0)

for i, img in enumerate(all_X):
    all_specs.append(freq_spec(img, HIGH_FREQ_THRESHOLD/100.0, add_noise=True))
    all_labels.append(1)

all_image_paths = np.concatenate([all_image_paths, all_image_paths])

all_image_paths = np.array(all_image_paths)
all_specs = np.array(all_specs)
all_labels = np.array(all_labels)

In [11]:
print("Quantidade das imagens:", all_specs.shape)
print("Quantidade dos paths de imagens:", all_image_paths.shape)
print("Exemplo dos labels (False = original, True = com ruído):", all_labels)

Quantidade das imagens: (3598, 224, 224)
Quantidade dos paths de imagens: (3598,)
Exemplo dos labels (False = original, True = com ruído): [0 0 0 ... 1 1 1]


In [12]:
X_train, X_test, y_train, y_test, paths_train, paths_test = train_test_split(
        all_specs, all_labels, all_image_paths,
        test_size=TEST_SIZE, 
        stratify=all_labels,
#       random_state=53
    )

In [13]:
def plot_specs(specs, title):
    total_images = len(specs)
    imgs_per_figure = 100
    cols = 5
    rows = (imgs_per_figure + cols - 1) // cols

    for start in range(0, total_images, imgs_per_figure):
        plt.figure(figsize=(cols * 3, rows * 3))
        end = min(start + imgs_per_figure, total_images)
        
        for i in range(start, end):
            plt.subplot(rows, cols, i - start + 1)
            plt.imshow(specs[i], cmap='gray')
            plt.title(f"{title} - freq spec: img {i+1}")
            plt.axis('off')
            
        plt.tight_layout()
        plt.show()

In [14]:
# Plotar todos os espectros de teste
#plot_specs(X_test, title="test")

In [15]:
#pausa de teste

In [16]:
# Função nomeada para conversão de canais
def expand_channels(x):
    return tf.stack([x[..., 0]]*3, axis=-1)

# Define as layers do modelo MobileNet
def model_mobilenet():
    inputs = Input(shape=(224, 224, 1))

    x = Lambda(expand_channels, name='channel_expander')(inputs)
    
    x = tf.keras.applications.mobilenet.preprocess_input(x)
    base_model = tf.keras.applications.MobileNet(
        weights='imagenet',
        include_top=False,
        input_tensor=x
    )
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    x = Dense(100, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [17]:
acc = []
jacc = []
f1 = []
prec = []
rec = []

# Configurar K-Fold com random_state fixo para reprodutibilidade
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=53)
kfold = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True)

# Verificar quais folds já foram completados
completed_folds = []
for f in range(1, NUM_FOLD + 1):
    model_path = output_dir + f'model_{nome}_fold_{f}.keras'
    if os.path.exists(model_path):
        completed_folds.append(f)
print(f"Folds concluídos: {completed_folds}")

fold_no = 1
histories = []
metrics = []

for train_idx, val_idx in kfold.split(X_train, y_train):
# Pular folds já concluídos
    if fold_no in completed_folds:
        print(f"\nPulando fold {fold_no} (já concluído)")
        fold_no += 1
        continue

    print(f'\nTreinando Fold {fold_no}/{NUM_FOLD}')

# Split dos dados
    X_train_fold = X_train[train_idx]
    y_train_fold = y_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train[val_idx]

# Pré-processamento final
    X_train_fold = np.expand_dims(X_train_fold, axis=-1)
    X_val_fold = np.expand_dims(X_val_fold, axis=-1)
    y_train_fold_cat = to_categorical(y_train_fold, 2)
    y_val_fold_cat = to_categorical(y_val_fold, 2)

# Criar novo modelo para cada fold
    model = model_mobilenet()

# Checkpoint com nome do fold
    checkpoint_filepath = output_dir + f'model_{nome}_fold_{fold_no}.keras'
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

# Calcula tempo (start)
    start_time = time.time()
    
# Treinar modelo
    history = model.fit(
        X_train_fold, y_train_fold_cat,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val_fold, y_val_fold_cat),
        callbacks=callbacks,
        verbose=1
    )

# Calcula tempo (end)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"\nO modelo demorou {training_time:.2f} segundos para treinar.")

# Coletar métricas e salvar modelo
    predictions = model.predict(X_val_fold)
    y_pred = np.argmax(predictions, axis=1)
    
    metrics.append({
        'fold': fold_no,
        'report': classification_report(y_val_fold, y_pred, output_dict=True, zero_division=0),
        'matrix': confusion_matrix(y_val_fold, y_pred)
    })
    
# Métricas de classificação (por fold)
    acc.append(accuracy_score(y_val_fold, y_pred))
    jacc.append(jaccard_score(y_val_fold, y_pred))
    f1.append(f1_score(y_val_fold, y_pred))
    prec.append(precision_score(y_val_fold, y_pred))
    rec.append(recall_score(y_val_fold, y_pred))

    print("Accuracy: "+ str(np.mean(acc)) + "+- " + str(np.std(acc)))
    print("Jaccard: "+ str(np.mean(jacc)) + "+- " + str(np.std(jacc)))
    print("Dice: "+ str(np.mean(f1)) + "+- " + str(np.std(f1)))
    print("Precision: "+ str(np.mean(prec)) + "+- " + str(np.std(prec)))
    print("Recall: "+ str(np.mean(rec)) + "+- " + str(np.std(rec)))

# Salvar métricas em um arquivo .txt
    metrics_filename = os.path.join(output_dir, f'metrics_{nome}_fold_{fold_no}.txt')
    with open(metrics_filename, 'w') as f:
        f.write(f"Fold {fold_no} Metrics:\n")
        f.write(f"Accuracy: {acc[-1]}\n")
        f.write(f"Jaccard Score: {jacc[-1]}\n")
        f.write(f"F1 Score: {f1[-1]}\n")
        f.write(f"Precision: {prec[-1]}\n")
        f.write(f"Recall: {rec[-1]}\n")
        f.write("\nClassification Report:\n")
        f.write(classification_report(y_val_fold, y_pred, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(confusion_matrix(y_val_fold, y_pred)))

# Limpeza de memória
    del model
    tf.keras.backend.clear_session()
    gc.collect()

# Salvar checkpoint a cada 3 folds
    if fold_no % 3 == 0:
        print(f"\nCheckpoint: Folds {fold_no-2}-{fold_no} concluídos")

    fold_no += 1

Folds concluídos: []

Treinando Fold 1/2


/tmp/ipykernel_1918/3785039319.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(
I0000 00:00:1747589570.291826    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747589570.292081    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747589570.292173    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1747589572.597213    1918 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel m

Epoch 1/75


I0000 00:00:1747589594.144839    1984 service.cc:146] XLA service 0x7efd40003e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747589594.144951    1984 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-05-18 14:33:14.767558: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-18 14:33:16.028078: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
2025-05-18 14:33:18.244232: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6710', 192 bytes spill stores, 192 bytes spill loads

I0000 00:00:1747589622.537433    1984 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.8153 - loss: 3.8972

...


Epoch 1: val_accuracy improved from -inf to 0.49969, saving model to Results_Streamlit/CNN-MobileNet (5%)/model_Yildirim_fold_1.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.8199 - loss: 3.8714 - val_accuracy: 0.4997 - val_loss: 13.8761
Epoch 2/75
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 1.9642
Epoch 2: val_accuracy improved from 0.49969 to 0.51822, saving model to Results_Streamlit/CNN-MobileNet (5%)/model_Yildirim_fold_1.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 8s 137ms/step - accuracy: 1.0000 - loss: 1.9590 - val_accuracy: 0.5182 - val_loss: 4.2763
Epoch 3/75
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 - loss: 1.0364
Epoch 3: val_accuracy improved from 0.51822 to 0.87153, saving model to Results_Streamlit/CNN-MobileNet (5%)/model_Yildirim_fold_1.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 1.0000 - loss: 1.0309 - val_accuracy: 0.8715 - val_loss: 0.9685
Epoch 4/75
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - 

2025-05-18 14:39:20.190895: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 324939776 exceeds 10% of free system memory.


51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step
Accuracy: 1.0+- 0.0
Jaccard: 1.0+- 0.0
Dice: 1.0+- 0.0
Precision: 1.0+- 0.0
Recall: 1.0+- 0.0

Treinando Fold 2/2


/tmp/ipykernel_1918/3785039319.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(


Epoch 1/75
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.8615 - loss: 3.6539
Epoch 1: val_accuracy improved from -inf to 0.50031, saving model to Results_Streamlit/CNN-MobileNet (5%)/model_Yildirim_fold_2.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 38s 406ms/step - accuracy: 0.8633 - loss: 3.6416 - val_accuracy: 0.5003 - val_loss: 14.5351
Epoch 2/75
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 1.6712
Epoch 2: val_accuracy did not improve from 0.50031
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 1.0000 - loss: 1.6607 - val_accuracy: 0.5003 - val_loss: 6.1274
Epoch 3/75
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 1.0000 - loss: 0.7740
Epoch 3: val_accuracy improved from 0.50031 to 0.65596, saving model to Results_Streamlit/CNN-MobileNet (5%)/model_Yildirim_fold_2.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 1.0000 - loss: 0.7693 - val_accuracy: 0.6560 - val_loss: 1.4305
Epoch 4/75
50/51 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 1.0000 -